# RECOMMENDER SYSTEMS

###### Refereces: 
<b>
<a href="https://towardsdatascience.com/how-to-build-a-simple-recommender-system-in-python-375093c3fb7d">How to build a Simple Recommender System in Python. ~ towardsdatascience.com</a>

<a href="https://towardsdatascience.com/beginners-recommendation-systems-with-python-ee1b08d2efb6">Beginner’s Recommendation Systems with Python (with the TMDB 5000 movies dataset) ~ towardsdatascience.com</a>

<a href="https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-recommendation-engine-python/">Comprehensive Guide to build a Recommendation Engine from scratch (in Python)  ~ Analytics Vidya</a>

<a href="https://realpython.com/build-recommendation-engine-collaborative-filtering/">Build a Recommendation Engine With Collaborative Filtering ~ RealPython.com</a>

<a href="https://www.datacamp.com/community/tutorials/recommender-systems-python">Recommender Systems in Python: Beginner Tutorial ~datacamp</a>

<a href="https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101">Recommender Systems in Python 101 ~ Kaggle</a>

<a href="https://www.google.com/search?q=how+to+build+a+great+recommnder+system+using+python">Google search for "how to build a great recommnder system using python"</a>

</b>

In [1]:
import sys
sys.path.append('C:/Users/nishita/exercises_udemy/MyTrials/tools')
from chinmay_tools import *

STEPS
* Load the dataset check the fields (recommender algo ALS - ALternating Least Squares needs few columns )
* 3 most important parameters are: userCol, itemCol, ratingCol in addition to maxIter and regParam
fields (For more details check https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html)
    * For input data we don't need 'features' vectorized column and hence no vectorAssembler needed.
    * Parameters to 'usrCol' and 'itemCol' serve as input. Compare these two against 'features' vector column in case of regression or classifiation algorithms.
    * 'ratingCol' is similar to label field (in regression or classification algorithms). Here the actual or observed rating is supplied which is given by the user (userCol=) to the movie (itemCol-)
* Process is the same .fit(train_data) on the ALS model that gets us the fitted_model i.e.als_model
* fitted_model.transform(test_Data) returns predictionsDF dataframe which gives us the predictions about the user rating to movies in test_data based on the training given to the machine on the training data movies.
* Evaluate by measuring metrics - RegressionEvaluator(metricName='rmse', ratingCol='rating', predictionCol='prediction').evaluate(predictionsDF)
* In this result "predictionsDF" dataframe the field 'prediction' tells the recommended ratings for respective movies
* To give recommendation to a specific user (i.e. unlabeled data),
    * To simulate unlabeled data we may filter the test dataframe to pick all the fields except the field used for 'ratingCol' i.e. 'rating' field) for that specific userid and let us store that object in a sDF called 'single_user'.
    * Transform this unlabeled data using trained_moded, i.e. do a fitted_model<b>.transform</b>(single_user) to get a prediction column in the result.
    * sort the result by decreasing order of 'prediction' values { result.orderBy('prediction', ascending='false') }
    * The movie with highest prediction is the recommended movie for the user.

## Refer: Consulting Project - Recommender Systems.ipynb

A new online food delivery company is trying to differentiate itself by recommending new meals to customers based off of other customers likings.

We need to build them a recommendation system whose final result should be in the form of a function that can take in a Spark DataFrame of a single customer's ratings for various meals and output their top 3 suggested meals.

* There is no data for meal preference by movie audience, which we need to fake / simulate
* We will assign some food preferences to audiences who saw the movies upto movieId 31, for rest we will set np.nan

In [20]:
import pandas as pd
import numpy as np

In [21]:
df_movie = pd.read_csv('Recommender_Systems/movielens_ratings.csv')

In [22]:
df_movie['mealskew'] = df_movie['movieId'].apply(lambda id: np.nan if id > 31 else id)

In [36]:
mealmap = { 
    0. : "Cheese Pizza",   
    1. : "Burrito",   
    2. : "Chicken Curry",   
    3. : "Spicy Chicken Nuggest",   
    4. : "Pretzels and Cheese Plate",   
    5. : "Hamburger",   
    6. : "Spicy Pork Sliders",  
    7. : "Nachos",  
    8. : "Chicken Chow Mein",  
    9. : "Taco Surprise",  
    10. : "Roasted Eggplant ",  
    11. : "Meatloaf",  
    12. : "Ceaser Salad",  
    13. : "Mandarin Chicken PLate",  
    14. : "Kung Pao Chicken",
    15. : "BBQ Ribs",  
    16. : "Fried Rice Plate",  
    17. : "Sushi Plate",  
    18. : "Pepperoni Pizza",  
    19. : "Cheesesteak Sandwhich",  
    20. : "Southwest Salad",  
    21. : "Lasagna",  
    22. : "Pulled Pork Plate",   
    23. : "Orange Chicken",
    24. : "Chili",  
    25.: "Roast Beef Sandwich",
    26. : "Spicy Beef Plate",  
    27. : "Salmon with Mashed Potatoes",  
    28. : "Penne Tomatoe Pasta",  
    29. : "Pork Sliders",  
    30. : "Vietnamese Sandwich",  
    31. : "Chicken Wrap"
#    ,  
#    np.nan: "Cowboy Burger"
}

In [38]:
df_movie['mealname'] = df_movie['mealskew'].map(mealmap, na_action='ignore')

In [40]:
df_movie

,movieId,rating,userId,mealskew,mealname
0,2,3.0,0,2.0,Chicken Curry
1,3,1.0,0,3.0,Spicy Chicken Nuggest
2,5,2.0,0,5.0,Hamburger
3,9,4.0,0,9.0,Taco Surprise
4,11,1.0,0,11.0,Meatloaf
...,...,...,...,...,...
1496,90,4.0,29,NaN,NaN
1497,93,1.0,29,NaN,NaN
1498,94,4.0,29,NaN,NaN
1499,97,1.0,29,NaN,NaN


In [41]:
df_movie.corr()

,movieId,rating,userId,mealskew
movieId,1.000000,0.036569,0.003267,1.000000
rating,0.036569,1.000000,0.056411,0.131044
userId,0.003267,0.056411,1.000000,0.017888
mealskew,1.000000,0.131044,0.017888,1.000000


In [43]:
df_movie.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0
mealskew,486.0,15.502058,9.250634,0.0,7.0,15.0,23.0,31.0


###### Now we have meal data with movie ratings 
* We assume that the 'rating' field is actually given by users for meal against the mealid identified as 'mealskew'.
* Let us use spark to do the following objective

We need to build them a recommendation system whose final result should be in the form of a function that can take in a Spark DataFrame of a single customer's ratings for various meals and output their top 3 suggested meals.

In [44]:
df_movie.to_csv('my_meal_info.csv', index=False)

In [45]:
from pyspark.sql import SparkSession

spark2 = SparkSession.builder.appName('reco_sys_2').getOrCreate()

In [46]:
sdf_meal = spark2.read.csv('my_meal_info.csv', inferSchema=True, header=True)

In [55]:
sdf_meal_no_na = sdf_meal.na.drop()

In [56]:
train_data, test_data = sdf_meal_no_na.randomSplit([0.7,0.3])

In [57]:
train_data.describe().show()
test_data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|           movieId|            rating|            userId|          mealskew|           mealname|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               349|               349|               349|               349|                349|
|   mean|15.744985673352435|  1.66189111747851|14.899713467048711|15.744985673352435|               null|
| stddev| 9.308614185803332|1.1143050407059607| 8.620926486224164| 9.308614185803332|               null|
|    min|                 0|               1.0|                 0|               0.0|           BBQ Ribs|
|    max|                31|               5.0|                29|              31.0|Vietnamese Sandwich|
+-------+------------------+------------------+------------------+------------------+-------------------+

+-------+------------------+-----------------

In [58]:
train_data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)
 |-- mealskew: double (nullable = true)
 |-- mealname: string (nullable = true)



In [59]:
from pyspark.ml.recommendation import ALS

* Assuming that the 'rating' was given by user for the meals identified by 'mealskew'

In [60]:
meal_model = ALS(userCol='userId', itemCol='mealskew', ratingCol='rating')

In [61]:
meal_model = ALS(userCol='userId', itemCol='mealskew', ratingCol='rating', maxIter=5, regParam=0.01)
meal_model = ALS(userCol='userId', itemCol='mealskew', ratingCol='rating')

###### Training gives error if null data is there, so I dropped the error data initially itself

In [63]:
trained_meal_model = meal_model.fit(train_data)

In [64]:
test_predictions = trained_meal_model.transform(test_data)
test_predictions.show()

+-------+------+------+--------+--------------------+----------+
|movieId|rating|userId|mealskew|            mealname|prediction|
+-------+------+------+--------+--------------------+----------+
|     31|   1.0|     5|    31.0|        Chicken Wrap| 0.5870483|
|     31|   1.0|    19|    31.0|        Chicken Wrap|  0.867401|
|     31|   1.0|     4|    31.0|        Chicken Wrap| 1.5316031|
|     31|   2.0|    25|    31.0|        Chicken Wrap|  1.453381|
|     31|   1.0|     0|    31.0|        Chicken Wrap| 1.1751199|
|     28|   1.0|     2|    28.0| Penne Tomatoe Pasta|  3.719898|
|     26|   1.0|     6|    26.0|    Spicy Beef Plate|0.91234356|
|     26|   1.0|    20|    26.0|    Spicy Beef Plate|0.94270444|
|     26|   2.0|    25|    26.0|    Spicy Beef Plate| 2.2946508|
|     27|   1.0|    13|    27.0|Salmon with Mashe...| 1.2962976|
|     27|   5.0|    23|    27.0|Salmon with Mashe...| 2.4648287|
|     27|   5.0|    11|    27.0|Salmon with Mashe...| 2.4038906|
|     12|   5.0|    28|  

In [65]:
from pyspark.ml.evaluation import RegressionEvaluator

In [66]:
eval2 = RegressionEvaluator(predictionCol='prediction', labelCol='rating', metricName='rmse')

In [67]:
rmse = eval2.evaluate(test_predictions)
print("Root-mean-square-error="+str(rmse))

Root-mean-square-error=1.330710338865221


In [69]:
test_predictions.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)
 |-- mealskew: double (nullable = true)
 |-- mealname: string (nullable = true)
 |-- prediction: float (nullable = false)



In [71]:
test_predictions.orderBy('prediction', ascending=False).show()

+-------+------+------+--------+--------------------+----------+
|movieId|rating|userId|mealskew|            mealname|prediction|
+-------+------+------+--------+--------------------+----------+
|     28|   1.0|     2|    28.0| Penne Tomatoe Pasta|  3.719898|
|     29|   5.0|     8|    29.0|        Pork Sliders| 3.3049076|
|     23|   5.0|    11|    23.0|      Orange Chicken| 2.8809085|
|     18|   4.0|    27|    18.0|     Pepperoni Pizza| 2.8471744|
|     10|   2.0|    24|    10.0|   Roasted Eggplant | 2.7924786|
|     14|   1.0|    12|    14.0|    Kung Pao Chicken|  2.590234|
|      8|   1.0|    12|     8.0|   Chicken Chow Mein| 2.5768778|
|     13|   4.0|    11|    13.0|Mandarin Chicken ...| 2.5267868|
|     22|   1.0|    18|    22.0|   Pulled Pork Plate| 2.5213304|
|     27|   5.0|    23|    27.0|Salmon with Mashe...| 2.4648287|
|     21|   3.0|    26|    21.0|             Lasagna| 2.4566429|
|      2|   1.0|    12|     2.0|       Chicken Curry| 2.4292555|
|     24|   5.0|    26|  

###### This orderBy clause is useless, but letus check just for curiosity

In [76]:
test_predictions.orderBy(test_predictions['rating'].desc(), test_predictions['prediction'].desc()).show()

+-------+------+------+--------+--------------------+----------+
|movieId|rating|userId|mealskew|            mealname|prediction|
+-------+------+------+--------+--------------------+----------+
|     29|   5.0|     8|    29.0|        Pork Sliders| 3.3049076|
|     23|   5.0|    11|    23.0|      Orange Chicken| 2.8809085|
|     27|   5.0|    23|    27.0|Salmon with Mashe...| 2.4648287|
|     24|   5.0|    26|    24.0|               Chili| 2.4173365|
|     27|   5.0|    11|    27.0|Salmon with Mashe...| 2.4038906|
|     23|   5.0|    26|    23.0|      Orange Chicken| 2.3865156|
|     30|   5.0|    11|    30.0| Vietnamese Sandwich| 2.2857206|
|      7|   5.0|     9|     7.0|              Nachos| 1.7047142|
|     12|   5.0|    28|    12.0|        Ceaser Salad| 1.3580644|
|     30|   5.0|    22|    30.0| Vietnamese Sandwich|  1.352006|
|     25|   5.0|     7|    25.0| Roast Beef Sandwich| 1.2237144|
|     17|   5.0|    17|    17.0|         Sushi Plate| 0.8033378|
|      7|   5.0|    26|  

###### There is something serious:
* There is a big gap between the 'rating' and 'prediction', we need to take care of that

# Refer: Recommender_Code_Along.ipynb

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark1 = SparkSession.builder.appName('reco_sys_1').getOrCreate()

In [4]:
sdf_mov = spark1.read.csv('Recommender_Systems/movielens_ratings.csv', inferSchema=True, header=True)

In [5]:
sdf_mov.describe().show()
sdf_mov.printSchema()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [6]:
(train_data, test_data) = sdf_mov.randomSplit([0.7,0.3])

In [7]:
from pyspark.ml.recommendation import ALS

In [8]:
als_model = ALS(maxIter=10, regParam=0.1, userCol='userId', itemCol='movieId', ratingCol='rating')

In [9]:
fitted_model = als_model.fit(train_data)

###### Now Evaluate the model performance by predicting on test _data and comparing the predictions against the rating column

In [10]:
test_predictions = fitted_model.transform(test_data)

In [11]:
test_predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   4.0|    12|0.45356953|
|     31|   1.0|     5|0.54007745|
|     31|   1.0|    19| 0.7227872|
|     31|   3.0|     7| 1.9319875|
|     31|   2.0|    25| 1.1689903|
|     31|   1.0|    24| 1.8058934|
|     31|   1.0|    29| 0.6315647|
|     31|   1.0|     0| 0.9767021|
|     85|   1.0|    28| 0.6391043|
|     85|   2.0|    20| 1.5787038|
|     85|   1.0|     5| 1.0694221|
|     85|   1.0|    23| 1.0222214|
|     65|   2.0|     3|  0.630873|
|     65|   1.0|    19| 1.1779414|
|     65|   5.0|    23| 1.6813419|
|     53|   2.0|    19| 2.7780185|
|     53|   1.0|     9| 2.6278894|
|     53|   1.0|    23| 1.1236882|
|     53|   1.0|    25| 0.7711124|
|     78|   1.0|    12|0.89054024|
+-------+------+------+----------+
only showing top 20 rows



<b>Cross validate the test predictions using an evaluator</b>

In [12]:
from pyspark.ml.evaluation import RegressionEvaluator

In [13]:
eval = RegressionEvaluator( predictionCol='prediction', labelCol='rating', metricName='rmse')

In [14]:
rmse = eval.evaluate(test_predictions)
printUnderlined('Root-mean-square-rror == '+str(rmse))

Root-mean-square-rror == 1.1005676909291717


In [15]:
test_data.columns
test_data

['movieId', 'rating', 'userId']

DataFrame[movieId: int, rating: double, userId: int]

###### Creating unlabeled data by dropping label i.e. 'rating' field for a specific user so that we can apply our model on this unlabled datato make predictions

In [16]:
user_11 = test_data.filter(test_data['userId']==11).select('movieId', 'userId')

In [17]:
user_11.show()

+-------+------+
|movieId|userId|
+-------+------+
|     16|    11|
|     18|    11|
|     21|    11|
|     25|    11|
|     38|    11|
|     39|    11|
|     59|    11|
|     62|    11|
|     64|    11|
|     66|    11|
|     99|    11|
+-------+------+



In [18]:
user11_recommendations = fitted_model.transform(user_11).orderBy('prediction')

In [19]:
user11_recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     18|    11| 3.6775835|
|     66|    11| 3.1345103|
|     16|    11| 2.8632846|
|     64|    11| 2.7695196|
|     62|    11| 1.9401884|
|     38|    11| 1.5345746|
|     21|    11| 1.3006377|
|     39|    11| 1.2993197|
|     99|    11| 1.0326141|
|     59|    11| 0.6672759|
|     25|    11|0.30686456|
+-------+------+----------+



###### The order of movie suggestion to the user (userid=11)  is in this order, first movie with movieid 18, then movie  66 and them movie 16